### 스포츠 센터의 탈퇴를 지도학습의 분류를 이용해서 예측

- 앞장에서 소개한 클러스터링을 통한 행동 분석은 사용 방법에 따라 많은 가능성이 있는 기술입니다. 행동 패턴을 분석할 수 있으면 어떤 고객이 탈퇴할지와 같은 예측도 어느 정도 정확하게 할 수 있기 때문에 탈퇴를 방지하기 위한 정책을 미리 준비하는 것도 가능합니다.

- 이 장에서는 이미 탈퇴한 회원과 계속해서 이용하는 회원의 데이터를 가지고 '의사결정 트리(Decision Tree)'라고 부르는 지도학습의 분류 알고리즘을 이용해서 탈퇴를 예측하는 흐름을 배웁니다.           
의사결정 트리는 간단한 수법이지만 알기 쉽게 원인 분석을 할 수 있기 때문에 현장에서 자주 사용됩니다.

고객의 소리) 상세 분석을 고려하여 생각해 보면 회원을 정착시키고 늘려가는 것보다 탈퇴를 막는 것이 중요한 것 같습니다. <u>탈퇴 회원이 왜 탈퇴했는지 알 수 있을까요?</u>

데이터셋)
1. use_log.csv : 센터 이용 이력(회원이 센터를 이용하면 시스템에 자동 입력 - 2018년 4월 ~ 2019년 3월의 1년분 데이터)     
2. customer_master.csv : 2019년 3월 말 시점의 회원 데이터(이전에 탈퇴한 회원 포함)             
3. class_master.csv : 회원 구분 데이터(종일, 주간, 야간)           
4. campaign_master.csv : 가입 시 행사 종류 데이터(입회비 유무 등)       
5. customer_use_merge_df.csv : 이전에 작성한 이용 이력을 포함한 고객 데이터               
6. monthly_use_df.csv : 이용 이력을 '연월/회원별'로 집계한 데이터 <b><u>New</u></b>


In [37]:
import pandas as pd 

customer_merge_use_df = pd.read_csv('./data/customer_use_merge_df.csv')
customer_merge_use_df.head()

monthly_use_df = pd.read_csv('./data/monthly_use_df.csv')
monthly_use_df.head()
# monthly_use_df.info()


,customer_id,use_month,count
0,AS002855,201804,4
1,AS002855,201805,5
2,AS002855,201806,5
3,AS002855,201807,5
4,AS002855,201808,3


In [25]:
customer_merge_use_df.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,2_야간,6000,2_일반,3.916667,4.0,6,1,1,2019-04-30,47


미래를 예측하기 위해 그 달과 1개월 전의 이용 이력만으로 데이터를 작성합니다.        
4장에서와 같이 과거 6개월분의 데이터로부터 이용 횟수를 예측하는 경우, 가입 5개월 이내인 회원의 탈퇴는 예측할 수 없습니다.   
불과 몇 개월 만에 그만둔 회원도 많기 때문에 과거 6개월분의 데이터를 이용한 예측은 의미가 없습니다.

In [ ]:
monthly_use_df['use_month'] == 201804
monthly_use_df['use_month'].dtype

# monthly_use_df.loc[monthly_use_df['use_month'] == 201804] # copying 문제로 loc을 쓰는 버릇** 필요

dtype('int64')

In [58]:
total_year_months = monthly_use_df['use_month'].unique()
total_year_months

data_for_ml = pd.DataFrame()
for i in range(1, len(total_year_months)) :
  tmp_pred = monthly_use_df.loc[monthly_use_df['use_month'] == total_year_months[i]]
  tmp_pred.rename(columns= {'count' : 'count_pred'}, inplace= True)
  tmp_before = monthly_use_df.loc[monthly_use_df['use_month'] == total_year_months[i - 1]]
  tmp_before.drop('use_month', axis= 1, inplace= True)
  tmp_before.rename(columns= {'count' : 'count_before'}, inplace= True)
  tmp = tmp_pred.merge(tmp_before, on= 'customer_id', how= 'left')
  data_for_ml = pd.concat([data_for_ml, tmp], axis= 0, ignore_index= True)

data_for_ml.head()

C:\Users\LGCARE\AppData\Local\Temp\ipykernel_22696\875998746.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_pred.rename(columns= {'count' : 'count_pred'}, inplace= True)
C:\Users\LGCARE\AppData\Local\Temp\ipykernel_22696\875998746.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.drop('use_month', axis= 1, inplace= True)
C:\Users\LGCARE\AppData\Local\Temp\ipykernel_22696\875998746.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,customer_id,use_month,count_pred,count_before
0,AS002855,201805,5,4.0
1,AS009373,201805,4,3.0
2,AS015233,201805,7,NaN
3,AS015315,201805,3,6.0
4,AS015739,201805,5,7.0


>> 탈퇴 회원의 데이터 작성(탈퇴 전월)

탈퇴 회원의 회원 정보와 사용 기록을 결합         
이 때, 탈퇴 신청 월을 기준으로 사용 기록을 결합(직전 월의 데이터가 예측에 중요)

2018년 8월  |  2018년 9월    |  2018년 10월              
탈퇴 신청    | 탈퇴 신청 완료  |      탈퇴                 
탈퇴 전월    | 탈퇴 월

- 왜 end_date 칼럼의 탈퇴 월이 아닌 탈퇴 전월의 데이터를 작성할까요?              
- 탈퇴를 예측하는 목적은 미연에 방지하는 것에 있습니다. 이 스포츠 센터에서는 월말까지 탈퇴 신청을 해야 다음 달 말에 탈퇴할 수 있습니다. 예를 들어, 2018년 9월 30일에 탈퇴(end_date)한 회원은 8월에 탈퇴 신청을 했기 때문에 9월의 데이터를 사용하는 것은 탈퇴를 방지할 수 없습니다. - <b><u>8월의 데이터(탈퇴 신청 달)를 사용해서 9월의 탈퇴(실제 탈퇴 달)를 예측해야 함</u></b>

- 탈퇴한 회원 추출
- end_date의 1개월 전을 계산하여 저장
- data_fol_ml(현재 + 직전월 사용 기록)와 customer_id, use_month를 키로 결합

In [ ]:
quit_customers_df = customer_merge_use_df.loc[customer_merge_use_df['is_deleted'] == 1]
quit_customers_df.reset_index(inplace= True, drop= True)
quit_customers_df.head()

from dateutil.relativedelta import relativedelta

quit_customers_df['withdrawl_application'] = None # 탈퇴 신청 월
quit_customers_df['end_date'] = pd.to_datetime(quit_customers_df['end_date'])

for i in range(len(quit_customers_df)) :
  quit_customers_df['withdrawl_application'].iloc[i] = quit_customers_df['end_date'].iloc[i] - relativedelta(months= 1) # 시각까지 포함된 값

quit_customers_df.head()

quit_customers_df['withdrawl_application'] = pd.to_datetime(quit_customers_df['withdrawl_application']).dt.strftime('%Y%m')
data_for_ml['use_month'] = data_for_ml['use_month'].astype(str)

# 'left' : 왼쪽의 모든 행을 유지하면서 오른쪽에서 조건이 맞는 것만 붙이고 없으면 NaN을 채운다는 뜻
quit_customer_monthly_use_df = data_for_ml.merge(quit_customers_df, how= 'left', left_on= ['customer_id', 'use_month'], right_on = ['customer_id', 'withdrawl_application'])

print(len(monthly_use_df)) # 36842 - 사용 로그 기준
print(len(quit_customer_monthly_use_df)) # 36842
quit_customer_monthly_use_df.head()


36842
33851


C:\Users\LGCARE\AppData\Local\Temp\ipykernel_22696\3016898579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quit_customers_df['withdrawl_application'] = pd.to_datetime(quit_customers_df['withdrawl_application']).dt.strftime('%Y%m')


,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period,withdrawl_application
0,AS002855,201805,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AS009373,201805,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AS015233,201805,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AS015315,201805,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AS015739,201805,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


결합한 데이터는 탈퇴한 회원의 탈퇴 전월의 데이터뿐이므로 결측치가 많습니다.                 
use_month는 있는데 withdrawl_application이 nan인 경우 - <b><u>사용 기록이 있는 회원이 그 달에 탈퇴하지 않은 것</u></b>         
사용 기록이 있는 경우는 전부 불러오는 merge

- quit_use_log : 탈퇴한 회원의 탈퇴 신청한 달의 사용 기록 & 고객 정보

In [ ]:
quit_use_log = quit_customer_monthly_use_df.dropna(subset= 'name') # name이 nan이면 탈퇴하지 않은 것(name부터의 열은 withdrawol_application 데이터프레임)
print(len(quit_use_log))
print(len(quit_use_log['customer_id'].unique()))
quit_use_log.head() # 탈퇴한 회원의 탈퇴 신청한 달의 사용 기록 & 고객 정보
# 어떤 특정 회원이 그만두기 전월의 상태를 나타내는 데이터


1104
1104


,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period,withdrawl_application
19,AS055680,201805,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,201805
57,AS169823,201805,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,201805
110,AS305860,201805,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,201805
128,AS363699,201805,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,201805
147,AS417696,201805,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,201805


>> 지속 회원의 데이터 작성

지속 회원은 탈퇴 월이 없기 때문에 어떤 연월의 데이터를 작성해도 됩니다. 따라서 지속 회원을 추출한 후 uselog 데이터에 결합하면 됩니다.

In [ ]:
continue_customer_df = customer_merge_use_df.loc[customer_merge_use_df['is_deleted'] == 0]
continue_customer_df.head()

continue_use_log = pd.merge(data_for_ml, continue_customer_df, on= 'customer_id', how= 'left')
print(len(continue_use_log)) # 33851 
continue_use_log = continue_use_log.dropna(subset=['name'])
print(len(continue_use_log)) # 29576

# monthly_use_df에는 있으나 continue_customer_df에는 없는 경우 > 탈퇴한 회원에 대한 데이터
# monthly_use_df에 하나의 customer_id에 use_month가 2018년 4월, 5월인 경우 고객 데이터가 각각 붙음

# 탈퇴 월이 없기 때문에 2018년 5월 A씨의 데이터나 2018년 12월 A씨의 데이터, 아무거나 지속 고객의 데이터로 사용 가능

33851
27422


In [ ]:
# 데이터 불균형 지속회원 : 29576 vs 탈퇴 회원 : 1104
# 지속 회원 > 회원당 1개로 언더샘플링
# 2018년 5월 A씨와 2018년 12월 A씨 중 하나만 선택

# 데이터를 섞고 중복을 제거 - 처음 나온 데이터를 가져옴
continue_use_log = continue_use_log.sample(frac= 1).reset_index(drop= True)
continue_use_log = continue_use_log.drop_duplicates(subset= 'customer_id')
print(len(continue_use_log)) # 2842
continue_use_log.head() 

2842


,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period
0,TS925945,201902,6,5.0,XXXXX,C03,M,2016-03-01,NaN,CA1,...,2_야간,6000.0,2_일반,4.416667,4.5,6.0,3.0,1.0,2019-04-30,37.0
1,HI020478,201901,9,8.0,XXXXX,C01,F,2017-12-01,NaN,CA3,...,0_종일,10500.0,1_입회비무료,6.833333,7.0,10.0,3.0,1.0,2019-04-30,16.0
2,OA248247,201812,7,8.0,XX,C01,M,2018-08-04,NaN,CA1,...,0_종일,10500.0,2_일반,9.125000,8.5,12.0,6.0,1.0,2019-04-30,8.0
3,HD679144,201812,9,11.0,XXXX,C01,M,2018-09-02,NaN,CA1,...,0_종일,10500.0,2_일반,8.857143,9.0,11.0,5.0,1.0,2019-04-30,7.0
4,GD079438,201903,6,5.0,XXXXXX,C02,F,2018-09-14,NaN,CA1,...,1_주간,7500.0,2_일반,7.428571,8.0,10.0,5.0,1.0,2019-04-30,7.0


>> 지속 회원 데이터, 탈퇴 회원 데이터 결합

In [102]:
predict_data = pd.concat([quit_use_log, continue_use_log], axis= 0, ignore_index= True)
print(len(predict_data))
predict_data.head()

3946


C:\Users\LGCARE\AppData\Local\Temp\ipykernel_22696\2503909923.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predict_data = pd.concat([quit_use_log, continue_use_log], axis= 0, ignore_index= True)


,customer_id,use_month,count_pred,count_before,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,regularity,calc_date,membership_period,withdrawl_application
0,AS055680,201805,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,201805
1,AS169823,201805,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,201805
2,AS305860,201805,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,201805
3,AS363699,201805,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,201805
4,AS417696,201805,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,201805


>> 예측할 달의 재적 기간 작성

시간적 요소가 들어간 데이터이므로 재적 기간과 같은 데이터를 변수로 이용하는 것이 좋은 접근이라고 할 수 있습니다.